In [1]:
import preprocessing
import utils
import implementations
import helpers

import numpy as np

In [2]:
y_tr, x_tr, _ = helpers.load_csv_data(path="resources/train.csv")

In [3]:
#y[np.where(y_tr == -1)] = 0

In [4]:
y_val, x_val, ids = helpers.load_csv_data(path="resources/test.csv")

In [5]:
#y_val[np.where(y_val == -1)] = 0

In [6]:
#x_tr, x_val, y_tr, y_val = utils.split_data(X, y, 0.8)

In [7]:
traning_groups = preprocessing.group_by_categories(x_tr, column=22)
validation_groups = preprocessing.group_by_categories(x_val, column=22)

In [8]:
x_tr = np.delete(x_tr, 22, axis=1)
x_val = np.delete(x_val, 22, axis=1)

# drop mass estimation, we can try to just make it binary
x_tr = np.delete(x_tr, 0, axis=1)
x_val = np.delete(x_val, 0, axis=1)

In [9]:
x_tr.shape

(250000, 28)

In [10]:
total_correct = 0
for i, (traning_group_idx, validation_group_idx) in enumerate(zip(traning_groups, validation_groups)):
    print('Category', i)
    print('-' * 30)

    x_tr_i, y_tr_i = x_tr[traning_group_idx], y_tr[traning_group_idx]
    x_val_i, _ = x_val[validation_group_idx], y_val[validation_group_idx]
    
    x_tr_i, x_val_i = preprocessing.preprocess_data(x_tr_i, x_val_i)
    x_tr_i = preprocessing.build_poly_coupling(x_tr_i, 12)
    x_val_i = preprocessing.build_poly_coupling(x_val_i, 12)
    '''weights, _ = implementations.reg_logistic_regression(
        y_tr_i, x_tr_i, 0, np.zeros(x_tr_i.shape[1]), max_iters=10000, gamma=0.2)'''
    weights, _ = implementations.least_squares(y_tr_i, x_tr_i)

    y_pred = np.array([-1 if x.T @ weights < 0 else 1  for x in x_val_i])
    #correct_predict = (y_pred == y_val_i).sum()
    #print("Group precision", correct_predict / len(y_val_i))

    #total_correct += correct_predict
    y_val[validation_group_idx] = y_pred
    print('*' * 30)

#print("Validation accuracy", total_correct / len(y_val))

Category 0
------------------------------
******************************
Category 1
------------------------------
******************************
Category 2
------------------------------
******************************
Category 3
------------------------------
******************************


In [11]:
(y_val == 1).sum() / len(y_val)

0.33502511271685453

In [12]:
y_val[np.where(y_val == 0)] = -1
print(y_val)

[-1. -1. -1. ...  1. -1. -1.]


In [13]:
helpers.create_csv_submission(ids, y_val, "lm1.csv")